## Baixar os dados

Neste caderno iremos baixar os dados a partir de três fontes:

1. [API TransfereGov módulo Transferências Especiais](https://docs.api.transferegov.gestao.gov.br/transferenciasespeciais/)
2. [API de dados da Câmara dos Deputados](https://dadosabertos.camara.leg.br/swagger/api.html)
3. [API de dados do Senado Federal](https://legis.senado.leg.br/dadosabertos/docs/index.html)

Após baixados, os dados serão salvos na pasta `dados`, para que não seja necessário baixá-los novamente.


Pacotes Python utilizados:

In [1]:
import os

import pandas as pd
import requests
from tqdm.notebook import tqdm

### API Transferegov

#### Parâmetros da API

In [2]:
LIMIT = 1000  # quantidade de reigstros por requisição
YEAR = 2024  # ano para filtrar os dados baixados
URL = "https://api.transferegov.gestao.gov.br/transferenciasespeciais/plano_acao_especial"

In [3]:
content = requests.get(
    URL,
    params={
        "ano_emenda_parlamentar_plano_acao": f"eq.{YEAR}",
        "limit": 10,
    },
    headers={
        "Prefer": "count=exact",
    }
)

In [4]:
content.headers
record_count = int(content.headers["Content-Range"].split("/")[-1])
print(f"Há {record_count} registros com os parâmetros de filtro informados.")

Há 10934 registros com os parâmetros de filtro informados.


In [5]:
data = content.json()
df = pd.DataFrame(columns=data[0].keys())

A API retorna os seguintes campos para o endpoint `plano_acao_especial`:

In [6]:
df.columns


Index(['id_plano_acao', 'codigo_plano_acao', 'ano_plano_acao',
       'modalidade_plano_acao', 'situacao_plano_acao',
       'cnpj_beneficiario_plano_acao', 'nome_beneficiario_plano_acao',
       'uf_beneficiario_plano_acao', 'codigo_banco_plano_acao',
       'codigo_situacao_dado_bancario_plano_acao', 'nome_banco_plano_acao',
       'numero_agencia_plano_acao', 'dv_agencia_plano_acao',
       'numero_conta_plano_acao', 'dv_conta_plano_acao',
       'nome_parlamentar_emenda_plano_acao',
       'ano_emenda_parlamentar_plano_acao',
       'codigo_parlamentar_emenda_plano_acao',
       'sequencial_emenda_parlamentar_plano_acao',
       'numero_emenda_parlamentar_plano_acao',
       'codigo_emenda_parlamentar_formatado_plano_acao',
       'codigo_descricao_areas_politicas_publicas_plano_acao',
       'descricao_programacao_orcamentaria_plano_acao',
       'motivo_impedimento_plano_acao', 'valor_custeio_plano_acao',
       'valor_investimento_plano_acao', 'id_programa'],
      dtype='object

#### Requisições à API

Para baixar os dados, será necessário iterar requisições nas página da API.

In [7]:
pages = []
for offset in tqdm(range(0, record_count, LIMIT)):
    content = requests.get(
        URL,
        params={
            "ano_emenda_parlamentar_plano_acao": f"eq.{YEAR}",
            "limit": LIMIT,
            "offset": offset,
        },
    )
    pages.extend(content.json())

df = pd.DataFrame(pages)
df

  0%|          | 0/11 [00:00<?, ?it/s]

,id_plano_acao,codigo_plano_acao,ano_plano_acao,modalidade_plano_acao,situacao_plano_acao,cnpj_beneficiario_plano_acao,nome_beneficiario_plano_acao,uf_beneficiario_plano_acao,codigo_banco_plano_acao,codigo_situacao_dado_bancario_plano_acao,...,codigo_parlamentar_emenda_plano_acao,sequencial_emenda_parlamentar_plano_acao,numero_emenda_parlamentar_plano_acao,codigo_emenda_parlamentar_formatado_plano_acao,codigo_descricao_areas_politicas_publicas_plano_acao,descricao_programacao_orcamentaria_plano_acao,motivo_impedimento_plano_acao,valor_custeio_plano_acao,valor_investimento_plano_acao,id_programa
0,69182,09032024-069182,2024,Especial,CIENTE,95990131000170,MUNICIPIO DE NOVA ITABERABA,SC,104,4.0,...,2855,22,202428550022,202428550022-Pedro Uczai,15-Urbanismo / 451-Infraestrutura Urbana,None,None,0.0,400000.0,21
1,64478,09032024-064478,2024,Especial,CIENTE,41522186000126,MUNICIPIO DE PASSAGEM FRANCA DO PIAUI,PI,001,4.0,...,4496,1,202444960001,202444960001-Jussara Lima,"13-Cultura / 391-Patrimônio Histórico, Artísti...",None,None,0.0,250000.0,21
2,64487,09032024-064487,2024,Especial,CIENTE,01612590000176,MUNICIPIO DE MILTON BRANDAO,PI,104,4.0,...,4496,1,202444960001,202444960001-Jussara Lima,"6-Segurança Pública / 181-Policiamento , 6-Seg...",None,None,0.0,250000.0,21
3,72493,09032024-072493,2024,Especial,CIENTE,18114264000131,MUNICIPIO DE ESPERA FELIZ,MG,104,4.0,...,3731,9,202437310009,202437310009-Misael Varella,"10-Saúde / 301-Atenção Básica , 10-Saúde / 302...",None,None,0.0,500000.0,21
4,73453,09032024-073453,2024,Especial,CIENTE,18114264000131,MUNICIPIO DE ESPERA FELIZ,MG,104,4.0,...,4057,3,202440570003,202440570003-Pinheirinho,"10-Saúde / 301-Atenção Básica , 10-Saúde / 302...",None,None,0.0,2000000.0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10929,74317,09032024-074317,2024,Especial,CIENTE,75368928000122,MUNICIPIO DE MAMBORE,PR,104,4.0,...,4313,1,202443130001,202443130001-Beto Preto,"15-Urbanismo / 451-Infraestrutura Urbana , 20-...",None,None,0.0,670000.0,21
10930,69119,09032024-069119,2024,Especial,CIENTE,14196042000154,TEOLÂNDIA,BA,104,4.0,...,3906,5,202439060005,202439060005-Alex Santana,8-Assistência Social / 244-Assistência Comunit...,None,None,1530737.0,2469263.0,21
10931,73476,09032024-073476,2024,Especial,CIENTE,18094870000132,MUNICIPIO DE SENHORA DOS REMEDIOS,MG,001,4.0,...,4057,3,202440570003,202440570003-Pinheirinho,"6-Segurança Pública / 181-Policiamento , 6-Seg...",None,None,0.0,500000.0,21
10932,72488,09032024-072488,2024,Especial,CIENTE,46362927000172,MUNICIPIO DE BROTAS,SP,104,4.0,...,3730,11,202437300011,202437300011-Miguel Lombardi,15-Urbanismo / 451-Infraestrutura Urbana,None,None,0.0,300000.0,21


#### Transformação

Vamos renomear a coluna que indica o nome do parlamentar para facilitar o tratamento.

In [8]:
df = df.rename(columns={"nome_parlamentar_emenda_plano_acao": "nome"})

#### Gravar os dados

In [9]:
os.makedirs("dados", exist_ok=True)

In [10]:
df.to_parquet("dados/transferencias-especiais.parquet")

### Câmara dos Deputados

Como os dados de deputados não são muitos, é possível baixar tudo em uma única requisição.

In [11]:
# URL da API da Câmara para obter partidos
url = "https://dadosabertos.camara.leg.br/api/v2/deputados"

# Requisição dos dados
response = requests.get(url, params={})
data = response.json()["dados"]

# Convertendo para DataFrame
df = pd.DataFrame(data)

# Renomeia a coluna de partido para ficar fácil cruzar os dados depois
df = df.rename(columns={"siglaPartido":"partido"})

df

,id,uri,nome,partido,uriPartido,siglaUf,idLegislatura,urlFoto,email
0,220593,https://dadosabertos.camara.leg.br/api/v2/depu...,Abilio Brunini,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MT,57,https://www.camara.leg.br/internet/deputado/ba...,dep.abiliobrunini@camara.leg.br
1,204379,https://dadosabertos.camara.leg.br/api/v2/depu...,Acácio Favacho,MDB,https://dadosabertos.camara.leg.br/api/v2/part...,AP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.acaciofavacho@camara.leg.br
2,220714,https://dadosabertos.camara.leg.br/api/v2/depu...,Adail Filho,REPUBLICANOS,https://dadosabertos.camara.leg.br/api/v2/part...,AM,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adailfilho@camara.leg.br
3,221328,https://dadosabertos.camara.leg.br/api/v2/depu...,Adilson Barroso,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SP,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adilsonbarroso@camara.leg.br
4,204560,https://dadosabertos.camara.leg.br/api/v2/depu...,Adolfo Viana,PSDB,https://dadosabertos.camara.leg.br/api/v2/part...,BA,57,https://www.camara.leg.br/internet/deputado/ba...,dep.adolfoviana@camara.leg.br
...,...,...,...,...,...,...,...,...,...
508,220558,https://dadosabertos.camara.leg.br/api/v2/depu...,Zé Trovão,PL,https://dadosabertos.camara.leg.br/api/v2/part...,SC,57,https://www.camara.leg.br/internet/deputado/ba...,dep.zetrovao@camara.leg.br
509,204517,https://dadosabertos.camara.leg.br/api/v2/depu...,Zé Vitor,PL,https://dadosabertos.camara.leg.br/api/v2/part...,MG,57,https://www.camara.leg.br/internet/deputado/ba...,dep.zevitor@camara.leg.br
510,160592,https://dadosabertos.camara.leg.br/api/v2/depu...,Zeca Dirceu,PT,https://dadosabertos.camara.leg.br/api/v2/part...,PR,57,https://www.camara.leg.br/internet/deputado/ba...,dep.zecadirceu@camara.leg.br
511,220592,https://dadosabertos.camara.leg.br/api/v2/depu...,Zezinho Barbary,PP,https://dadosabertos.camara.leg.br/api/v2/part...,AC,57,https://www.camara.leg.br/internet/deputado/ba...,dep.zezinhobarbary@camara.leg.br


#### Gravar os dados

In [12]:
os.makedirs("dados", exist_ok=True)

In [13]:
df.to_parquet("dados/deputados.parquet")

### Senado Federal

Os dados de senadores são em menor quantidade ainda, vamos baixar tudo de uma vez.

In [14]:
# URL da API para buscar senadores
url = "https://legis.senado.leg.br/dadosabertos/senador/lista/atual.json"

# Fazer a requisição à API
response = requests.get(url)

# Parse do JSON
data = response.json()

# Extrair os dados relevantes
senadores = data['ListaParlamentarEmExercicio']['Parlamentares']['Parlamentar']
nomes = []
partidos = []
ufs = []

for senador in senadores:
    nome = senador['IdentificacaoParlamentar']['NomeParlamentar']
    partido = senador['IdentificacaoParlamentar']['SiglaPartidoParlamentar']
    uf =  senador['IdentificacaoParlamentar']['UfParlamentar']
    nomes.append(nome)
    partidos.append(partido)
    ufs.append(uf)

# Criar um DataFrame para exibir os dados
#df = pd.DataFrame({'Nome': nomes, 'Partido': partidos})
df = pd.DataFrame({'nome': nomes, 'partido': partidos, 'uf': ufs})

df

,nome,partido,uf
0,Alan Rick,UNIÃO,AC
1,Alessandro Vieira,MDB,SE
2,Ana Paula Lobato,PDT,MA
3,Angelo Coronel,PSD,BA
4,Astronauta Marcos Pontes,PL,SP
...,...,...,...
76,Wellington Fagundes,PL,MT
77,Weverton,PDT,MA
78,Wilder Morais,PL,GO
79,Zenaide Maia,PSD,RN


#### Gravar os dados

In [15]:
os.makedirs("dados", exist_ok=True)

In [16]:
df.to_parquet("dados/senadores.parquet")